In [1]:
import pandas as pd
import numpy as np
import os

In [ ]:
issue = pd.read_csv("../../data/google_22_output_multi.csv")

In [3]:
issue.head(2)

,text,id,ISSUE215,ISSUE10,ISSUE30,ISSUE40,ISSUE212,ISSUE12,ISSUE16,ISSUE209,...,ISSUE91,ISSUE56,ISSUE53,ISSUE90,ISSUE65,ISSUE45,ISSUE60,ISSUE58,ISSUE22,ISSUE32
0,'30 MASSEY- HARRIS PAID FOR BY DAVID TRONE FOR...,CR01131952237317193729__aws_ocr_video_text|CR0...,0,0,0,0,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1,"'HER WAY' """"КОТЕК CROSSED THE LINE"""" OPB.ORG N...",CR01885433262180925441__aws_ocr_video_text|CR0...,0,0,0,1,1,0,0,0,...,0,0,0,0,0,1,0,0,0,0


In [4]:
# Function to get column names where value is 1 for the last 25 columns
def get_active_features(row):
    active_features = issue.columns[-25:][row.iloc[-25:] == 1]
    return ','.join(active_features)

In [5]:
# Apply the function to each row
issue['issue'] = issue.apply(get_active_features, axis=1)

In [6]:
# Split ids
issue['id'] = issue['id'].str.split('|')
# "Un-deduplicate", or "Re-hydrate", in WMP lingo
issue = issue.explode('id')
# Split into ad id and field
issue_ids = issue['id'].str.split('__', expand = True)
issue_ids.columns = ['ad_id', 'field']
issue = pd.concat([issue, issue_ids], axis = 1)
issue = issue.drop(labels = ['id'], axis = 1)
issue = issue[['ad_id', 'field', 'issue']]

In [7]:
issue.rename(columns={'field': 'issue_field', 'issue': 'issue_class'}, inplace=True)

In [8]:
# Replace various representations of missing values with NaN
missing_values = ['', None, 'NA', 'missing']
issue['issue_class'] = issue['issue_class'].replace(missing_values, np.nan)
issue = issue.dropna(subset=['issue_class'])

In [9]:
agg_issue = issue.groupby('ad_id').agg({'issue_field': lambda x: '|'.join(x),
                                       'issue_class': lambda x: '|'.join(x)}).reset_index()

In [10]:
agg_issue.head(5)

,ad_id,issue_field,issue_class
0,CR00000257354440376321,aws_ocr_video_text|google_asr_text,"ISSUE10,ISSUE212|ISSUE10,ISSUE212"
1,CR00000354386341527553,ad_text,ISSUE10
2,CR00001130641550737409,aws_ocr_video_text|google_asr_text,"ISSUE212|ISSUE30,ISSUE212"
3,CR00001435481149538305,google_asr_text,"ISSUE91,ISSUE45"
4,CR00002421743079653377,ad_title|ad_text,ISSUE22|ISSUE22


In [11]:
agg_issue.shape

(47914, 3)

In [ ]:
agg_issue.to_csv("../../data/g2022_output_processed.csv", index = False)